# 1. Data fetching and cleaning
In this notebook we are going to fetch and perform an initial exploration of the dataset for the Git track of the Hércules Challenge. This dataset consists of 50 GitHub repositories.

## Setup
As always, we will begin the notebook by starting the logging system and importing some constants defined in the "\_\_init\_\_.py" file:

In [1]:
%run __init__.py

We will also define an auxiliary function to print the empty columns of the dataframes:

In [2]:
def print_empty_cols(df):
    for col in df.columns:
        print(col)
        print('-' * len(col))
        res = df[df[col] == ''].index
        print(f"{len(res)} articles have no value for column {col}")
        print(res)
        print('\n')


Finally, we will impor the bokeh library to show the charts in the notebook, and we will import the BokehHistogram class from the _herc\_common_ library to show our results:

In [3]:
from bokeh.io import output_notebook

output_notebook()

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Loading BokehJS ...

In [4]:
from herc_common import BokehHistogram

hist = BokehHistogram(color_fill="mediumslateblue", color_hover="slateblue", bins=25)

## Getting the repository URLs

The URL for every repository of the Git dataset is stored in the '_data/repo\_urls.txt_' file. First of all, we will be loading all the urls from that file into a list:

In [5]:
REPO_URLS_FILE = 'repo_urls.txt'

with open(os.path.join(DATA_DIR, REPO_URLS_FILE), 'r') as f:
    repo_urls = [line.rstrip('\n') for line in f]

len(repo_urls)

50

In [6]:
repo_urls[0]

'https://github.com/cmungall/LIRICAL/'

## Parsing the data

Now that the URLs of every repository have been saved, we can start calling the [GitHub API](https://developer.github.com/v3/) to obtain information about each repo. Since the API has a limit of 50 requests per hour for non-authorized requests, and we will be making about 200 requests to fetch all the information, we will need to make use of a personal token to make calls to the API. More information about what a personal token is, and how to create one, can be accessed through the [following link](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token). In the next cell we will be asking for the token:

In [7]:
import getpass


try:
    from secret import GITHUB_TOKEN
except ModuleNotFoundError:
    GITHUB_TOKEN = getpass.getpass("Introduce your personal access token to acces the GitHub API: ")


We will now import a series of classes and functions that will be used to fetch information about a given repo and convert it to an instance of the _GitHubRepoData_ class. More information about these functions and classes can be accessed at the _src_ package:

In [8]:
from src import GitHubIssue, GitHubRepoData, parse_repo_url

Finally, we will be creating a list of GitHubRepoData instances with information about every repository from the dataset:

In [9]:
from tqdm import tqdm

git_dataset = []
pbar = tqdm(repo_urls)
for url in pbar:
    pbar.set_description(f"Processing repository: {url}")
    git_dataset.append(parse_repo_url(url, GITHUB_TOKEN))


Processing repository: https://github.com/mikel-egana-aranguren/FAIRifier:   8%|▊         | 4/50 [00:33<07:09,  9.33s/it]    

mikel-egana-aranguren-FAIRifier-d69d6a4/extensions/fairifier-rdf-extension/scripts/externals/angular.min.js


Processing repository: https://github.com/mikel-egana-aranguren/elda:  10%|█         | 5/50 [01:56<23:32, 31.39s/it]     

mikel-egana-aranguren-elda-e9bd9d3/elda-assets/src/main/webapp/openlayers/proj4js-combined.js
mikel-egana-aranguren-elda-e9bd9d3/elda-assets/src/main/webapp/velocity/js/vendor/proj4js-combined.js


Processing repository: https://github.com/fanavarro/DnaCompress:  24%|██▍       | 12/50 [03:59<08:27, 13.36s/it]                          

...

Processing repository: https://github.com/pjotrp/genenetwork2:  34%|███▍      | 17/50 [05:18<06:35, 11.99s/it]    

pjotrp-genenetwork2-86d885f/wqflask/wqflask/static/new/javascript/box.js


Processing repository: https://github.com/micheldumontier/php-lib:  68%|██████▊   | 34/50 [17:10<06:21, 23.87s/it]                      

micheldumontier php-lib


Processing repository: https://github.com/pauldevos/Basketball_Analytics: 100%|██████████| 50/50 [23:31<00:00, 28.24s/it]         


## Creating a dataframe

The instances created before provide a _to\_dict_ method that can be used to convert the class to a Python dict. This dict can be used to easily create a pandas DataFrame. This DataFrame will be used from now on to explore and interact with the dataset:

In [10]:
import pandas as pd

df = pd.DataFrame([repo.to_dict() for repo in git_dataset])
df.head()

,gh_id,name,description,owner_name,languages,readme_text,issues_text,commits_text,comments_text
0,216602979,LIRICAL,LIkelihood Ratio Interpretation of Clinical Ab...,cmungall,"{'Java': 492423, 'FreeMarker': 13149, 'Python'...",LIRICAL. LIkelihood Ratio Interpretation of C...,,Merge pull request #442 from TheJacksonLaborat...,note that the Jannovar dependency does not...
1,199330464,wikidata_ontomatcher,Matches ontology classes against wikidata,cmungall,"{'Prolog': 14691, 'Makefile': 1472, 'Dockerfil...",Match an ontology to Wikidata. This applicati...,Will help with #1 and with https://github.com/...,Adding skos:altLabel\n\nhttps://github.com/cmu...,
2,253207181,ro-crate-ruby,"A Ruby gem for creating, manipulating and read...",markwilkinson,"{'Ruby': 52724, 'HTML': 1319}","ro-crate-ruby. This is a WIP gem for creating,...",,Update LICENSE\nBump version\nTidy up and chec...,*\n * Expands the tree to the target element a...
3,212556220,Misc_Training_scripts,A place for me to keep various miscellanelous ...,markwilkinson,"{'Shell': 15815, 'Ruby': 9445}",Misc_Training_scripts. A place for me to keep ...,,added new cool 3-federated query\nfinished edi...,
4,155879756,FAIRifier,A tool to make data FAIR,mikel-egana-aranguren,"{'Java': 3514431, 'JavaScript': 967765, 'HTML'...",Dependencies: Java 8. Apache Ant. Building. in...,,Merge pull request #16 from Shamanou/developme...,*\n * This class contains all the code shared ...


## Data cleaning and feature engineering

First of all, we will be taking an initial look to the values from the dataset:

In [11]:
df.loc[:, df.columns != 'gh_id'].describe()

,name,description,owner_name,languages,readme_text,issues_text,commits_text,comments_text
count,50,43,50,50,50,50,50,50
unique,50,43,27,50,49,6,50,30
top,php-lib,Tableau-based reasoner for ALCQ description logic,be-sanders,"{'C#': 15985, 'CSS': 649, 'JavaScript': 34}",,,Changed line numbers to gray.\nRemoved tm's lo...,
freq,1,1,2,1,2,45,1,21


As we can see above, altough all the repository names are unique, the other columns have some repeated values. Those repeated values could be empty or null values, so we are going to check if that is the case:

In [12]:
df[df.isnull().any(axis=1)]

,gh_id,name,description,owner_name,languages,readme_text,issues_text,commits_text,comments_text
11,57412597,hemodonacion,None,fanavarro,"{'Perl': 97370, 'R': 36211}",Lost in Translation. Structure. This repositor...,,delete files\nnew execution\nReadme updated\nr...,
23,161862375,biohack18,None,leechuck,{'Groovy': 14573},biohack18.,,initial commit\nInitial commit,
24,171842501,biosample_jsonld,None,inutano,"{'Ruby': 9183, 'Shell': 3171, 'Dockerfile': 226}",BioSample records in JSON-LD. BioSample is a d...,,mkdir\nremove postgres lib\nremove comment whi...,
29,151696606,JavaTermiteStarter,None,SciBiteLabs,{'Java': 16959},JavaTermiteStarter. Set of basic code to get y...,,Create README.md\nRefactoring\nRefactoring\nMe...,*\n * This class was created by simon on 03/09...
39,42526998,pythonontologysearch,None,jamesmalone,"{'JavaScript': 194124, 'Python': 5748, 'HTML':...",,,remove test message\nChange call to ontology_i...,****\n****\n****\n****\n****\n****\n****\n****...
40,238953196,my-react-form,None,twhetzel,"{'JavaScript': 35896, 'CSS': 6855, 'HTML': 1271}",React Form Demos. Description. Demo project to...,,remove file App.css\nadd more example forms\na...,\n manifest.json provides metadata used w...
41,157134523,python-ecology-lesson-es-1,None,agbeltran,"{'Jupyter Notebook': 4579723, 'HTML': 60649, '...",Canal de Slack en español Lección de Data Car...,,Update fechas-progreso.md\nMerge pull request ...,"\n <div class=""col-md-6"">\n <h3 id=""syllab..."


We can see from the output above that there are 7 repositories which do not have a description. We are going to replace those null values by an empty string:

In [13]:
df.fillna(value="", inplace=True)

Now, we are going to see know how many columns have an empty value:

In [14]:
print_empty_cols(df)

gh_id
-----
0 articles have no value for column gh_id
Int64Index([], dtype='int64')


name
----
0 articles have no value for column name
Int64Index([], dtype='int64')


description
-----------
7 articles have no value for column description
Int64Index([11, 23, 24, 29, 39, 40, 41], dtype='int64')


owner_name
----------
0 articles have no value for column owner_name
Int64Index([], dtype='int64')


languages
---------
0 articles have no value for column languages
Int64Index([], dtype='int64')


readme_text
-----------
2 articles have no value for column readme_text
Int64Index([8, 39], dtype='int64')


issues_text
-----------
45 articles have no value for column issues_text
Int64Index([ 0,  2,  3,  4,  5,  6,  8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20,
            21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
            39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
           dtype='int64')


commits_text
------------
0 articles have no value for column commits_text


/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Most of the repositories (45 out of 50) don't have any issues, and 3 of them don't have a readme.

Finally, we are going to join both the description and the readme of each repository into a new column, and remove all extra spaces from that column:

In [15]:
import re

def clean(text):
    return re.sub('\s+', ' ', text).strip()

df['full_text'] = df["description"] + ". " + df["readme_text"]
            #". " + df["comments_text"] + \
            #". " + df["commits_text"]
empty_idx = df[df['full_text'] == '. '].index
df.loc[empty_idx, 'full_text'] = df.loc[empty_idx, 'commits_text']
df['full_text_cleaned'] = df['full_text'].apply(lambda x: clean(x))
df['full_text_cleaned'].loc[0][:500]

'LIkelihood Ratio Interpretation of Clinical AbnormaLities. LIRICAL. LIkelihood Ratio Interpretation of Clinical AbnormaLities. LIRICAL is designed to provide clincially interpretable computational analysis of phenotypic abnormalities (encoded using the Human Phenotype Ontology), optionally combined with an analysis of variants and genotypes if a VCF file is provided with the results of diagnostic gene panel, exome, or genome sequencing. Detailed documentation is available This is a useful websit'

## Initial exploration

To finish with the contents of this notebook, we will make an initial exploration of the dataset.

### Text length

We are going to add a new column to the DataFrame with the length in number of characters of each repo's full text:

In [16]:
df['num_chars_text'] = df['full_text_cleaned'].apply(lambda x: len(x))
df['num_chars_text'].describe()

count       50.000000
mean      2348.480000
std       3235.137547
min         12.000000
25%        337.250000
50%       1751.500000
75%       2906.000000
max      19317.000000
Name: num_chars_text, dtype: float64

We can see that the average number of characters in the readme + description is about 2471, and the maximum length is 20382 characters. However, 75% of the repositories have a number of characters lower than 3027.

We are going to plot this distribution and save it to disk:

In [17]:
GIT_HIST_COLUMN = "num_chars_text"
GIT_HIST_TITLE = "Readme + Description length distribution"
GIT_HIST_XLABEL = "Readme and description length (# of characters)"
GIT_HIST_YLABEL = "Number of repositories"

hist.load_plot(df, GIT_HIST_COLUMN, GIT_HIST_TITLE,
          GIT_HIST_XLABEL, GIT_HIST_YLABEL, True)

In [18]:
hist.save_plot(os.path.join(NOTEBOOK_1_RESULTS_DIR, '1_Repo_text_length.svg'))

### Languages used

Finally, we are going to also explore the most used programming languages for each repository.

We will begin by creating an auxiliary function that will create an horizontal bar chart with the given data:

In [19]:
from bokeh.io import show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category20b_20
from bokeh.plotting import figure

def plot_horizontal_bar_chart(x_data, y_data, title, tooltip_x, tooltip_y,
                              sort=True, color_palette=Category20b_20):
    sorted_y_data = sorted(y_data, key=lambda x: x_data[y_data.index(x)]) if sort else y_data
    source = ColumnDataSource(data=dict(y_data=y_data, x_data=x_data, color=color_palette))
    p = figure(y_range=sorted_y_data, x_range=(0, max(x_data) * 1.1), plot_height=750, title=title,
               toolbar_location='right')
    p.hbar(y='y_data', right='x_data', height=0.7, color='color', legend_field="y_data",
           fill_alpha=0.75, hover_fill_alpha=1.0, source=source)
    p.ygrid.grid_line_color = None
    p.legend.orientation = "vertical"
    p.legend.location = "bottom_right"
    p.add_tools(HoverTool(tooltips=[(tooltip_y, "@y_data"), (tooltip_x, "@x_data")],
                          point_policy="follow_mouse"))

    show(p, notebook_handle=True)



Now, we can create new dataframe with each different programming language used in the dataset and the number of bytes belonging to that language for each repo:

In [20]:
languages_df = pd.DataFrame(df['languages'].values.tolist()).fillna(value=0, inplace=False)
languages_df.head()

,Java,FreeMarker,Python,Prolog,Makefile,Dockerfile,Shell,Ruby,HTML,JavaScript,...,Raku,TSQL,PowerShell,Game Maker Language,Web Ontology Language,PLpgSQL,Gherkin,Common Lisp,ActionScript,C#
0,492423.0,13149.0,849.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,14691.0,1472.0,700.0,278.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52724.0,1319.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,15815.0,9445.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3514431.0,0.0,0.0,0.0,0.0,0.0,28836.0,2559.0,333450.0,967765.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


By making use of the previously defined function and the new DataFrame we can plot the top 15 languages with the most number of bytes written in each repository:

In [21]:
from bokeh.palettes import Category20_15

NUM_LANGUAGES = 15

languages_sum = languages_df.sum().sort_values(ascending=False)
num_bytes = languages_df.sum()[:NUM_LANGUAGES]

plot_horizontal_bar_chart(list(languages_sum)[:NUM_LANGUAGES],
                          list(languages_sum.keys())[:NUM_LANGUAGES],
                          "Languages with the most number of bytes",
                          "Number of bytes", "Language",
                          color_palette=Category20_15)

We can see that most repositories have code written in Jupyter Notebooks, JavaScript and Java.

Although the number of bytes is an interesting measure, some languages tend to have a bigger repository size by nature. In the following cell we are going to select the most prominent language for each repository and plot the most used languages in the dataset:

In [22]:
from bokeh.palettes import Category20_10

most_used_languages = languages_df.idxmax(axis=1).value_counts()[:10]
plot_horizontal_bar_chart(list(most_used_languages),
                          list(most_used_languages.keys()),
                          "Top 10 most used languages",
                          "Number of repositories", "Language",
                          color_palette=Category20_10)

With this new measure the top order has changed a bit. Both JavaScript and Jupyter Notebooks remain in the top 5, but they have fallen some positions to both Java and Python.

## Saving the dataframe

Finally, we are going to serialize the dataframe so we can load it later on in the following notebooks:

In [23]:
GIT_DF_FILE_PATH = os.path.join(NOTEBOOK_1_RESULTS_DIR, 'git_dataframe.pkl')

df.to_pickle(GIT_DF_FILE_PATH)